In [1]:
# Importing the StringIO module.
from bs4 import BeautifulSoup

# import the requests library 
import requests

import pandas as pd

import re

#Inserting stock data into SQL database using sqlachemy
from sqlalchemy import create_engine

#Defining headers with User agent to establish requests
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36',
'X-Requested-With': 'XMLHttpRequest'
}

from selenium import webdriver
from selenium.webdriver.firefox.options import Options
#from selenium.webdriver.chrome.options import Options
import time
from bs4 import BeautifulSoup
import csv

import pandas as pd
import numpy as np

In [2]:

#lst = ['SBIN','TCS','HCLTECH','ROSSARI','PIDILITIND']

#Function to scrape Marketsmith stock fundamental data
def marketsmith_scrape(lst):
    for x in range(0,len(lst)):
        options = Options()
        options.add_argument('--headless')

        #Firefox browser webdriver
        driver = webdriver.Firefox(options=options)


        #Google chrome webdriver
        #options = webdriver.ChromeOptions()
        #driver = webdriver.Chrome(options=options)

        #Base Url to fetch stock fundamental data through request


        url = "https://marketsmithindia.com/mstool/eval/{}/evaluation.jsp#/".format(lst[x])
        print(url)
    
        driver.get(url)
        #time.sleep(2)
        #soup = BeautifulSoup(driver.page_source, 'lxml')
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        

        #Populating summary columns for individual stocks from Marketsmith evaluation pages
        if x==0:
            stock_detail_header = soup.find_all('div', class_ = "details")

        print(format(lst[x]),">>>" ,stock_detail_header)

        stock_detail_value = soup.find_all('div', class_ = "value")
        print(format(lst[x]),">>>",stock_detail_value)
        
        cols = []
        
        
        #Generating columns through iteration
        
        for y in range(0,len(stock_detail_header)):
            cols.append(stock_detail_header[y].get_text().strip())
            
        #Creating a new column for stock wise segregation
        cols.insert(0,"Stock Name")
        
        print(y,"columns","--->>>",cols)
        
        
        #share_summary = pd.DataFrame(columns= cols)
        
        
        #Feeding an empty Dataframe with its respective column values through iterating summary_data list
        #Creating an empty Dataframe to feed the column names through iterating summary_cols list

        summary_data_values = []
        

        #Generating columns data through iteration
        for z in range(0,len(stock_detail_value)):
            print(stock_detail_value[z].get_text().strip())
            summary_data_values.append(stock_detail_value[z].get_text().strip())
        
        #Adding stock name as a new column to every data rows for identification.
        summary_data_values.insert(0,lst[x])       
        
        print(z,"summary_data_values","--->>>",summary_data_values)    
        
        #Creating base data frame to hold data for every stock
        share_summary = pd.DataFrame(columns= cols,data = [summary_data_values])
        
        #Removing duplicate rows to avoid discrepency
        share_summary = share_summary.loc[:,~share_summary.columns.duplicated()]
        
        #Creating final Data frame one time and from base dataframe structure
        if x==0:
            #share_summary = pd.DataFrame(columns= cols,data = [summary_data_values])
            
            all_share_summary = pd.DataFrame(columns= [share_summary])
        print ("Inside x loop",x)
            
        #Adding individual stock data values through every iteration    
        print("Final Datatypes","<<<<<<<<->>>>>>>>",all_share_summary.dtypes)
        all_share_summary = all_share_summary._append(share_summary,ignore_index=True)
        
        #del(stock_detail_header,stock_detail_value,share_summary)
        summary_data_values.clear()


        #Deleting objects cache to contents to store data in next iteration
        soup.decompose()
        
        print("Next Iteration")
    
    #Data cleaning - Dropping unncessary columns
    all_share_summary.drop(list(all_share_summary)[0:23], axis=1,inplace = True)
    
    #Renaming column name for SQL table processing
    all_share_summary.rename(columns = {'Stock Name' : 'stock_name', 'Market Capitalization' : 'market_capitalization', 'Sales' : 'sales', \
        'Shares in Float' : 'shares_in_float', 'No of Funds' : 'no_of_funds', 'Shares held by Funds' : 'shares_held_by_funds', 'Yield' : 'yield', \
        'Book Value' : 'book_value', 'U/D Vol Ratio' : 'u/d_vol_ratio', 'LTDebt/Equity' : 'ltdebt_equity', 'Alpha' : 'alpha', 'Beta' : 'beta', \
        'Master Score' : 'master_score', 'EPS Rating' : 'eps_rating', 'Price Strength' : 'price_strength', 'Acc/Dis Rating' : 'buyers_demand', \
        'Group Rank' : 'group_rank', 'EPS Growth Rate' : 'eps_growth_rate', 'Earnings Stability' : 'earnings_stability', 'P/E Ratio' : 'pe_ratio', \
        '5-Year P/E Range' : '5years_pe_range', 'Return on Equity' : 'return_on_equity', 'Cash Flow (INR)': 'cash_flow'},
        inplace = True)
    
    #Changing datatypes for the columns to their appropraite type for further processing
    all_share_summary.master_score = all_share_summary.master_score.astype('Int32')
    all_share_summary.eps_rating = all_share_summary.eps_rating.astype('Int32')
    all_share_summary.price_strength = all_share_summary.price_strength.astype('Int32')
    all_share_summary.earnings_stability = all_share_summary.earnings_stability.astype('Int32')
    all_share_summary.pe_ratio = all_share_summary.pe_ratio.astype('Int32')
    
    #Inserting stock data into SQL database using sqlachemy
    
    #Defining table name on MSSQL server to locate the data
    table_name = "market_smith_stock_eval"
    
    sql_engine = create_engine('mssql+pyodbc://' + "DESKTOP-EQ55Q8H" + '/' + "NSEBhavcopy" + '?trusted_connection=yes&driver=SQL+Server')

    db_conn= sql_engine.connect()

    #DB Actions to load data from Pandas Dataframe to MSSQL
    try:
        all_share_summary.to_sql(table_name, db_conn, if_exists= 'replace',index= False)
    except Exception as ex:
        print(ex)
    else:
        print('Marketsmith Stock Evalulation data successfully inserted into MS SQL table-{}'.format(table_name))
    finally:
        db_conn.close()
    
    return all_share_summary


In [3]:
lst = ['MRF','SBIN','ROSSARI','HCLTECH','TCS']

#Calling webscraping function
marketsmith_scrape(lst)

https://marketsmithindia.com/mstool/eval/MRF/evaluation.jsp#/
MRF >>> [<div class="details"> Market Capitalization</div>, <div class="details"> Sales </div>, <div class="details"> Shares in Float </div>, <div class="details"> No of Funds </div>, <div class="details"> Shares held by Funds </div>, <div class="details"> Market Capitalization </div>, <div class="details"> Sales </div>, <div class="details"> Shares in Float </div>, <div class="details"> No of Funds </div>, <div class="details"> Shares held by Funds </div>, <div class="details"> Yield </div>, <div class="details"> Book Value </div>, <div class="details"> U/D Vol Ratio </div>, <div class="details"> LTDebt/Equity </div>, <div class="details"> Alpha </div>, <div class="details"> Beta </div>, <div class="details"> Master Score </div>, <div class="details"> EPS Rating </div>, <div class="details"> Price Strength </div>, <div class="details"> Acc/Dis Rating </div>, <div class="details"> Group Rank </div>, <div class="details"> EPS

,stock_name,market_capitalization,sales,shares_in_float,no_of_funds,shares_held_by_funds,yield,book_value,u/d_vol_ratio,ltdebt_equity,...,eps_rating,price_strength,buyers_demand,group_rank,eps_growth_rate,earnings_stability,pe_ratio,5years_pe_range,return_on_equity,cash_flow
0,MRF,"INR 56,997.75 Cr",INR 24184.02 Cr,30.53 L,630 (+1.29 %),12.86 L (-1.78 %),0.13%,3.89x,1.3,6%,...,85,49,C,113 of 197,N/A,43,27,21-\n 74,6%,5698.11
1,SBIN,"INR 7,40,742.61 Cr",INR 445313.40 Cr,383.76 Cr,1.32 K (+4.52 %),309.48 Cr (-0.14 %),1.41%,2.46x,1.1,151%,...,76,68,B+,85 of 197,63%,78,13,9-\n 350,17%,-102.36
2,ROSSARI,"INR 4,081.10 Cr",INR 1202.91 Cr,1.77 Cr,69 (+2.99 %),1.20 Cr (-1.92 %),0.06%,4.31x,1.1,3%,...,79,21,C+,30 of 197,N/A,61,42,45-\n 104,10%,20.61
3,HCLTECH,"INR 3,70,452.75 Cr",INR 48118.00 Cr,105.83 Cr,1.42 K (+4.02 %),93.99 Cr (+0.18 %),2.31%,9.4x,0.5,0%,...,58,32,E,120 of 197,4%,24,32,16-\n 44,30%,49.89
4,TCS,"INR 13,92,493.51 Cr",INR 202359.00 Cr,101.31 Cr,1.72 K (+4.05 %),84.55 Cr (+3.59 %),1.2%,19.18x,0.8,N.A,...,62,36,C-,120 of 197,4%,21,32,25-\n 41,60%,101.20
